In [1]:
ITER = '0'

In [2]:
import pandas as pd
import os.path

DATASET_FOLDER = '/Users/jilljenn/Desktop/GCFM/datasets/'

In [3]:
people.shape

NameError: name 'people' is not defined

In [34]:
truth = pd.read_csv(os.path.join(DATASET_FOLDER, 'duck/DUCKgt.txt'), sep='\t', names=('item_id', 'truth'))

In [35]:
truth.shape

(108, 2)

In [5]:
people = pd.read_csv(os.path.join(DATASET_FOLDER, 'duck/DUCKlabel.txt'), sep=' ', names=('item_id', 'worker', 'label'))

In [9]:
df = people.merge(truth, on='item_id')

In [10]:
df.head()

,item_id,worker,label,truth
0,36618,896,0,0
1,36618,866,1,0
2,36618,39,0,0
3,36618,175,1,0
4,36618,1721,1,0


In [11]:
import numpy as np

df['correct'] = (df['label'] == df['truth']).astype(np.int32)

In [12]:
encode = {}
for feature in {'item_id', 'worker'}:
    encode[feature] = dict(zip(df[feature].unique(), range(10000)))
df['item'] = df['item_id'].map(encode['item_id'])
df['user'] = df['worker'].map(encode['worker'])

In [13]:
df.head()

,item_id,worker,label,truth,correct,item,user
0,36618,896,0,0,1,0,0
1,36618,866,1,0,0,0,1
2,36618,39,0,0,1,0,2
3,36618,175,1,0,0,0,3
4,36618,1721,1,0,0,0,4


In [14]:
# Save dataset into the KTM folder
# df[['user', 'item', 'correct']].to_csv('/Users/jilljenn/code/ktm/data/duck/data.csv', index=None)

In [15]:
# Save config into the KTM folder
import yaml

with open('/Users/jilljenn/code/ktm/data/duck/config.yml', 'w') as f:
    yaml.safe_dump({
        'nb_users': 1 + int(df['user'].max()),
        'nb_items': 1 + int(df['item'].max())
    }, f)

## Subsets

In [39]:
ITER = '1'

In [76]:
test = {}
for index in range(1):
    subset = pd.read_csv(os.path.join(DATASET_FOLDER, 'duck/subgolden/DUCKsubgoldengt.{}.txt'.format(index)), sep='\t', names=('item', 'truth'))
    known_items = set(subset['item'])
    i_train = df.query('item_id in @known_items').index
    test[index] = df.query('item_id not in @known_items').copy()
    i_test = test[index].index
    # Save indices for train and test into the KTM folder
    # np.save('/Users/jilljenn/code/ktm/data/duck/i_train{}.npy'.format(index), i_train)
    # np.save('/Users/jilljenn/code/ktm/data/duck/i_test{}.npy'.format(index), i_test)
    # Load predictions from KTM
    test[index]['pred'] = np.load('/Users/jilljenn/code/ktm/data/duck00/y_pred{}.npy'.format(index))
    test[index]['confidence'] = abs(test[index]['pred'] - 0.5)
    test[index]['round_pred'] = np.round(test[index]['pred'])
    test[index]['one'] = 1
    test[index]['recommend'] = (1 - test[index]['round_pred']) * (1 - test[index]['label']) + test[index]['round_pred'] * test[index]['label']
test[0].head()

,item_id,worker,label,truth,correct,item,user,confidence0,pred,confidence,round_pred,one,recommend
0,36618,896,0,0,1,0,0,0.0,0.331393,0.168607,0.0,1,1.0
1,36618,866,1,0,0,0,1,0.0,0.528325,0.028325,1.0,1,1.0
2,36618,39,0,0,1,0,2,0.0,0.899391,0.399391,1.0,1,0.0
3,36618,175,1,0,0,0,3,0.0,0.437513,0.062487,0.0,1,0.0
4,36618,1721,1,0,0,0,4,0.0,0.398165,0.101835,0.0,1,0.0


In [77]:
scores = test[0].groupby(['item_id', 'recommend']).sum()

In [78]:
max_sum = scores.groupby('item_id').max().rename(columns={key: 'max_sum_' + key for key in {'pred', 'confidence', 'one'}})

In [79]:
scores = scores.join(max_sum[['max_sum_pred', 'max_sum_confidence', 'max_sum_one']], on='item_id')

In [80]:
for feature in {'pred', 'confidence', 'one'}:
    scores['{}_choice'.format(feature)] = scores[feature] == scores['max_sum_{}'.format(feature)]

In [81]:
scores.head()

worker  label  truth  correct  item  user  confidence0  \
item_id recommend                                                           
11573   0.0         10518      3      7        3   175   106          7.0   
        1.0         47116     24     32       24   800   635         32.0   
11575   0.0          7009      2      4        2   108    96          4.0   
        1.0         50625     24     35       24   945   645         35.0   
11577   0.0         40660      5     27        5  1485   502          0.0   

                        pred  confidence  round_pred  one  max_sum_pred  \
item_id recommend                                                         
11573   0.0         3.823618    0.825912         4.0    7     22.591976   
        1.0        22.591976    8.324232        24.0   32     22.591976   
11575   0.0         2.129866    0.571186         2.0    4     23.159849   
        1.0        23.159849   10.508641        24.0   35     23.159849   
11577   0.0        17.803861    5.778003        22.0   27     17.803861   

                   max_sum_confidence  max_sum_one  confidence_choice  \
item_id recommend                                                       
11573   0.0                  8.324232           32              False   
        1.0                  8.324232           32               True   
11575   0.0                 10.508641           35              False   
        1.0                 10.508641           35               True   
11577   0.0                  5.778003           27               True   

                   one_choice  pred_choice  
item_id recommend                           
11573   0.0             False        False  
        1.0              True         True  
11575   0.0             False        False  
        1.0              True         True  
11577   0.0              True         True

In [69]:
#ITER = '0'
ITER = '2'
for feature in {'pred', 'confidence', 'one'}:
    pred = scores.query('{}_choice == True'.format(feature)).reset_index()[['item_id', 'recommend']].rename(columns={'recommend': feature + ITER})
    truth = truth.merge(pred, on='item_id')

In [70]:
truth.head(), truth.shape

(   item_id  truth  confidence1_x  one1_x  pred1_x  confidence1_y  one1_y  \
 0    36618      0            0.0     0.0      0.0            0.0     0.0   
 1    11619      1            1.0     1.0      1.0            1.0     1.0   
 2    36622      0            0.0     0.0      0.0            0.0     0.0   
 3    36623      0            0.0     0.0      0.0            0.0     0.0   
 4    36624      0            0.0     0.0      0.0            0.0     0.0   
 
    pred1_y  confidence2  one2  pred2  
 0      0.0          0.0   0.0    0.0  
 1      1.0          1.0   1.0    1.0  
 2      0.0          0.0   0.0    0.0  
 3      0.0          0.0   0.0    0.0  
 4      0.0          0.0   0.0    0.0  , (81, 11))

In [73]:
ITER = '2'

In [75]:
sum(truth['confidence1_x'] == truth['confidence2']), len(truth)

(81, 81)

In [72]:
from sklearn.metrics import accuracy_score, roc_auc_score

for feature in {'pred', 'confidence', 'one'}:
    print('acc', feature, '=', accuracy_score(truth['truth'], truth[feature + ITER]))
    print('auc', feature, '=', roc_auc_score(truth['truth'], truth[feature + ITER]))

acc confidence = 0.8024691358024691
auc confidence = 0.7925336597307221
acc one = 0.8024691358024691
auc one = 0.7925336597307221
acc pred = 0.8024691358024691
auc pred = 0.7925336597307221


In [29]:
df.shape

(4212, 7)

In [30]:
df = df.merge(truth[['item_id', 'confidence' + ITER]], on='item_id')

In [32]:
df.head()

,item_id,worker,label,truth,correct,item,user,confidence0
0,36618,896,0,0,1,0,0,0.0
1,36618,866,1,0,0,0,1,0.0
2,36618,39,0,0,1,0,2,0.0
3,36618,175,1,0,0,0,3,0.0
4,36618,1721,1,0,0,0,4,0.0


In [33]:
df.shape

(3159, 8)

In [81]:
df['correct'] = (df['label'] == df['confidence0_x']).astype(np.int32)

In [82]:
df.head()

,item_id,worker,label,truth,correct,item,user,confidence0_x,confidence0_y,correct0
0,36618,896,0,0,1,0,0,0.0,0.0,1
1,36618,866,1,0,0,0,1,0.0,0.0,0
2,36618,39,0,0,1,0,2,0.0,0.0,1
3,36618,175,1,0,0,0,3,0.0,0.0,0
4,36618,1721,1,0,0,0,4,0.0,0.0,0


In [38]:
df[['user', 'item', 'correct']].to_csv('/Users/jilljenn/code/ktm/data/duck00/data.csv', index=None)

In [37]:
df.shape

(3159, 8)